In [1]:
import os
import clip
import torch
from torchvision.datasets import CIFAR100
import math
from pytorch_lightning.loggers import WandbLogger
from tqdm import tqdm
import numpy as np
import torch
import pytorch_lightning as pl
from filelock import FileLock
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
import os
import torchvision
import clip
from clip_lt.utils.labels_names import labels_names
import torch.nn as nn
from PIL import Image
import time
from clip_lt.utils.labels_names import labels_names
import glob
from clip_lt.blip.models.blip_itm import blip_itm
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = "cuda" if torch.cuda.is_available() else "cpu"
image_size = 384


model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth'

model = blip_itm(pretrained=model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device=device)
pass

load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_retrieval_coco.pth


In [ ]:
# dataset_dir_path = '/Volumes/black_ssd/datasets/imagenet_lt/'
dataset_dir_path = '/Users/rotemisraeli/Documents/datasets/imagenet_lt/'

TRANSFORM_IMG = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225] )
])

text_features = torch.load('../text_features2.pt')
text_features = (text_features / text_features.norm(dim=-1, keepdim=True)).t().to(device)
logit_scale = (nn.Parameter(torch.ones([]) * np.log(1 / 0.07))).exp().to(device)

In [ ]:
good = 0.0
bad = 0.0
with torch.no_grad():
    tqdm_ = tqdm(range(1000))
    for label in tqdm_:
        for image_name in glob.glob(f'{dataset_dir_path}val/{label}/*'):
            image = preprocess(Image.open(image_name)).unsqueeze(0).to(device)
            torchvision.utils.save_image(image,'test2.png')
            image_features = clip_model.encode_image(image)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
            logits_per_image = logit_scale * image_features @ text_features
            probs = logits_per_image.softmax(dim=-1)
            predicted = torch.argmax(probs)
            if predicted == label:
                good += 1.0
            else:
                bad += 1.0
            tqdm_.desc = f'{100*good/(good+bad)}'




In [2]:
out = model(torch.zeros(1,3,image_size,image_size),'test')